In [12]:
import pandas as pd
import re

In [13]:
# Constants
labelled_column_name = 'Labelled'

### Comparative binding to wild type vs mutant SARS-CoV-2 NP antigens by different epitope groups

In [14]:
antigens_by_epitope_groups = pd.read_csv("antigens_by_epitope_groups.csv", sep='\t')
antigens_by_epitope_groups.dropna(inplace=True)
antigens_by_epitope_groups = antigens_by_epitope_groups.reset_index()
antigens_by_epitope_groups

,index,blank,group
0,1,NP1501,IA
1,2,NP1507,IA
2,3,NP1514,IA
3,4,NP1516,IA
4,5,NP1517,IA
5,7,NP1526,IB
6,8,X190,IB
7,9,X200,IB
8,10,X201,IB
9,11,NP1512,IB/II


In [15]:
antigen_names = antigens_by_epitope_groups['blank'].tolist()

### Cross inhibition raw data

In [16]:
ci_raw_data = pd.read_csv("cross_inhibitor_raw_data.csv", sep='\t')
ci_raw_data.dropna(inplace=True)
ci_raw_data_filtered = ci_raw_data.loc[ci_raw_data[labelled_column_name].isin(antigen_names)]
ci_raw_data_filtered.reset_index(level=0, drop=True, inplace=True)
ci_raw_data_filtered = ci_raw_data_filtered.drop('Neat', axis=1)

In [17]:
ci_raw_data_filtered

,Labelled,NP1501*,NP1502*,NP1503*,NP1508*,NP1510*,NP1514*,NP1516*,NP1517*,NP1518*,...,X202*,X211*,X213*,X215*,X217*,X220*,X221*,X223*,X233*,X271*
0,NP1501,0.449,0.715,1.0664,1.0248,1.136,0.2600,0.4172,0.268,0.8512,...,0.5316,1.2267,1.2519,0.6400,1.0224,1.189,0.8856,1.4616,0.769,1.228
1,NP1502,0.893,0.425,0.3360,0.1960,0.349,0.3625,0.6650,0.450,1.0000,...,0.6492,1.2087,1.2753,0.5500,1.0593,1.203,0.2979,1.4184,0.809,0.407
2,NP1503,0.768,0.309,0.0680,0.0520,0.095,0.3050,0.7126,0.408,0.9888,...,0.6420,1.1970,1.2582,0.6075,1.0791,1.215,0.1035,1.4704,0.930,0.109
3,NP1507,0.422,0.856,0.7216,0.6088,0.785,0.1825,0.4256,0.154,1.0352,...,0.6432,1.2213,1.2672,0.5125,1.0215,1.244,0.6624,1.4168,1.018,0.843
4,NP1508,0.732,0.388,0.1456,0.0848,0.190,0.3450,0.8456,0.411,1.1344,...,0.6492,1.2474,1.2699,0.5575,1.0089,1.151,0.2070,1.3760,1.052,0.233
5,NP1510,0.781,0.382,0.2152,0.1056,0.233,0.4950,0.7826,0.527,1.0000,...,0.6744,1.1547,1.1934,0.5750,0.9513,1.117,0.2331,1.4200,0.942,0.220
6,NP1512,0.790,0.789,0.8760,0.7504,0.979,0.7350,1.0150,0.737,0.8160,...,0.6012,1.1826,1.0251,0.7425,0.8532,1.123,0.6534,1.2544,1.018,0.627
7,NP1514,0.448,0.822,1.0968,1.0088,1.189,0.2475,0.4858,0.253,1.0208,...,0.6816,1.2384,1.2240,0.8075,1.0413,1.332,0.8712,1.4616,0.772,0.964
8,NP1516,0.385,1.034,0.9832,0.9304,1.053,0.1775,0.3052,0.152,0.9504,...,0.5916,1.2078,1.1844,0.5925,1.0116,1.261,0.7920,1.4944,0.878,0.974
9,NP1517,0.425,0.644,0.7952,0.7304,0.885,0.1550,0.2758,0.079,1.1504,...,0.5016,1.1646,1.1601,0.5075,1.0179,1.245,0.7209,1.4376,0.882,1.011


### Cross inhibition sorted by homology

In [18]:
labelled_column_location = ci_raw_data_filtered.columns.get_loc(labelled_column_name)

In [19]:
# Get groups and insert
groups = ci_raw_data_filtered.apply(
    lambda row: antigens_by_epitope_groups.loc[antigens_by_epitope_groups['blank'] == row[labelled_column_name], 'group'].iloc[0], axis=1
)
ci_raw_data_filtered.insert(labelled_column_location+1, 'group', groups)

# Get sorting order and insert
sorting_order = ci_raw_data_filtered.apply(
    lambda row: antigens_by_epitope_groups.loc[antigens_by_epitope_groups['blank'] == row[labelled_column_name], 'index'].iloc[0], axis=1
)
ci_raw_data_filtered.insert(
    len(ci_raw_data_filtered.columns), 'sorting_order', sorting_order)


In [20]:
# Group by group and sort rows by sorting_order
ci_raw_data_filtered.groupby("group")
ci_raw_data_filtered = ci_raw_data_filtered.set_index('group', append=True)
ci_raw_data_filtered = ci_raw_data_filtered.swaplevel(0, 1)

ci_raw_data_filtered = ci_raw_data_filtered.reset_index(level=1, drop=True)
ci_raw_data_filtered = ci_raw_data_filtered.set_index('Labelled', append=True)

ci_raw_data_filtered = ci_raw_data_filtered.sort_values(
    by='sorting_order', key=lambda x: x.astype(int))


In [21]:
def ci_column_compare(x):
    index = antigens_by_epitope_groups.loc[
                antigens_by_epitope_groups['blank'].str.contains(x[:-1]), 'index']
    return index.values[0] if len(index)>0 else float('inf')

cols = ci_raw_data_filtered.columns.tolist()
cols.sort(key=ci_column_compare)

ci_raw_data_filtered = ci_raw_data_filtered[cols]

In [22]:
ci_raw_data_filtered

NP1501*  NP1514*  NP1516*  NP1517*   X200*  X201*  NP1521*  \
group   Labelled                                                               
IA      NP1501      0.449   0.2600   0.4172    0.268  0.8073  1.055    0.596   
        NP1507      0.422   0.1825   0.4256    0.154  0.4491  0.963    0.583   
        NP1514      0.448   0.2475   0.4858    0.253  0.8910  1.038    0.626   
        NP1516      0.385   0.1775   0.3052    0.152  0.6606  1.063    0.455   
        NP1517      0.425   0.1550   0.2758    0.079  0.5103  1.121    0.414   
IB      NP1526      0.492   0.2200   0.4074    0.210  0.2556  0.206    0.453   
        X190        0.442   0.3275   0.4452    0.239  0.0846  0.168    0.349   
        X200        0.509   0.3900   0.4466    0.437  0.1746  0.401    0.345   
        X201        0.443   0.3875   0.4802    0.426  0.1917  0.309    0.389   
IB/II   NP1512      0.790   0.7350   1.0150    0.737  0.5643  0.439    0.638   
        NP1521      0.538   0.4425   0.6342    0.574  0.7209  1.267    0.107   
II      NP1502      0.893   0.3625   0.6650    0.450  0.2295  0.185    0.535   
        NP1503      0.768   0.3050   0.7126    0.408  0.1053  0.144    0.570   
        NP1508      0.732   0.3450   0.8456    0.411  0.1674  0.170    0.550   
        NP1510      0.781   0.4950   0.7826    0.527  0.1836  0.193    0.661   
        NP1522      0.669   0.4725   0.7476    0.574  0.1584  0.177    0.441   
        NP1525      1.003   0.7650   0.9506    0.670  0.1656  0.205    0.672   
        NP1520      0.669   0.3150   0.5866    0.406  0.1314  0.186    0.431   
        NP3701      0.966   0.3100   0.4844    0.437  0.2151  0.621    0.283   
        X221        0.646   0.5600   0.6818    0.603  0.1692  0.320    0.484   
        X271        0.487   0.5375   0.4648    0.538  0.0936  0.161    0.399   
        NP3708      0.676   0.1850   0.3920    0.175  0.1449  0.263    0.624   
II/III? NP1528      1.103   0.3075   0.5474    0.445  0.1827  0.541    0.273   
IIIA    X202        0.745   0.6025   0.6776    0.692  0.7614  1.419    0.504   
        X218        0.424   0.5475   0.4956    0.653  0.7713  1.038    0.390   
        NP1518      0.517   0.2425   0.5502    0.340  0.6174  1.527    0.394   
        NP1527      0.719   0.4050   0.5600    0.594  0.6273  1.392    0.464   
IIIB    X32         0.559   0.8450   0.8204    0.797  1.3140  1.114    0.626   
        X41         0.659   0.7275   0.6944    0.695  0.8379  1.329    0.405   
        X155        0.504   0.5025   0.5250    0.562  0.5616  1.312    0.397   
        X212        0.712   0.8700   0.9436    0.872  1.0278  1.759    0.664   
        X213        0.484   0.6600   0.5978    0.644  0.8559  1.243    0.386   
        X217        0.476   0.5325   0.4494    0.609  0.6588  1.160    0.305   
        X223        0.694   0.7450   0.7616    0.849  0.7128  0.846    0.539   
        X224        0.895   0.8475   0.9408    0.882  0.7650  1.050    0.562   
        X233        0.629   0.7125   0.7728    0.830  0.8082  1.101    0.434   
        NP1524      0.604   0.7425   0.9926    0.833  0.8055  1.358    0.675   
        NP3715      0.605   0.4100   0.7686    0.554  0.4437  1.164    0.612   
IVA     NP3706      0.565   0.4550   0.5334    0.589  0.5580  1.889    0.508   
IVB     X211        0.640   0.7650   0.8428    0.836  0.8856  1.474    0.598   
IVC     X215        0.615   0.6050   0.4928    0.665  0.7641  1.476    0.339   
V       X220        0.508   0.4375   0.4844    0.590  0.7641  1.553    0.393   
        X275        0.437   0.5575   0.3836    0.708  0.4392  1.646    0.378   

                  NP1502*  NP1503*  NP1508*  ...   X217*   X223*  X233*  \
group   Labelled                             ...                          
IA      NP1501      0.715   1.0664   1.0248  ...  1.0224  1.4616  0.769   
        NP1507      0.856   0.7216   0.6088  ...  1.0215  1.4168  1.018   
        NP1514      0.822   1.0968   1.0088  ...  1.0413  1.4616  0.772   
        NP1516      1.034   0.9832   0.930